# backtest_strategy 라이브러리 사용 예시

이 노트북은 새로 만든 backtest_strategy 라이브러리의 사용법을 보여줍니다.

## 1. 라이브러리 임포트 및 데이터 준비

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
from google.oauth2 import service_account

# backtest_strategy 라이브러리 임포트
from backtest_strategy import *

# matplotlib 한글 설정
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False

In [2]:
# BigQuery 클라이언트 초기화 (노트북의 기존 코드 재사용)
def get_bigquery_client():
    service_account_path = "/Users/cg01-piwoo/my_quant/access_info/data/quantsungyong-663604552de9.json"
    credentials = service_account.Credentials.from_service_account_file(
        service_account_path,
        scopes=["https://www.googleapis.com/auth/cloud-platform"]
    )
    return bigquery.Client(credentials=credentials, project=credentials.project_id)

client = get_bigquery_client()

# 데이터 로드 함수 (간단한 버전)
def get_stock_data_with_indicators(ticker, start_date=None, end_date=None):
    date_conditions = []
    if start_date:
        date_conditions.append(f"JSON_EXTRACT_SCALAR(r.dates_array[OFFSET(i.pos)], '$') >= '{start_date}'")
    if end_date:
        date_conditions.append(f"JSON_EXTRACT_SCALAR(r.dates_array[OFFSET(i.pos)], '$') <= '{end_date}'")
    
    date_filter = f"AND {' AND '.join(date_conditions)}" if date_conditions else ""
    
    query = f"""
    WITH raw_data AS (
      SELECT 
        ticker,
        -- 가격 데이터
        JSON_EXTRACT_ARRAY(data, '$.dates') AS dates_array,
        JSON_EXTRACT_ARRAY(data, '$.open') AS open_array,
        JSON_EXTRACT_ARRAY(data, '$.high') AS high_array,
        JSON_EXTRACT_ARRAY(data, '$.low') AS low_array,
        JSON_EXTRACT_ARRAY(data, '$.close') AS close_array,
        JSON_EXTRACT_ARRAY(data, '$.volume') AS volume_array,
        
        -- ADX 관련 지표
        JSON_EXTRACT_ARRAY(data, '$.adx_14_values') AS adx_14_values_array,
        JSON_EXTRACT_ARRAY(data, '$.pdi_14_values') AS pdi_14_values_array,
        JSON_EXTRACT_ARRAY(data, '$.mdi_14_values') AS mdi_14_values_array,
        
        -- Chaikin Oscillator 관련 지표 (수정된 필드명)
        JSON_EXTRACT_ARRAY(data, '$.chaikin_oscillator') AS chaikin_oscillator_array,
        JSON_EXTRACT_ARRAY(data, '$.chaikin_9_signal_line') AS chaikin_signal_array,
        
        -- 배열 길이
        ARRAY_LENGTH(JSON_EXTRACT_ARRAY(data, '$.close')) AS array_length
        
      FROM 
        `quantsungyong.finviz_data.stock_data_with_indicators`
      WHERE 
        ticker = '{ticker}'
    ),
    indices AS (
      SELECT r.ticker, pos
      FROM raw_data r,
      UNNEST(GENERATE_ARRAY(0, r.array_length - 1)) AS pos
    )
    SELECT 
      -- 날짜와 가격 데이터
      JSON_EXTRACT_SCALAR(r.dates_array[OFFSET(i.pos)], '$') AS date,
      CAST(JSON_EXTRACT_SCALAR(r.open_array[OFFSET(i.pos)], '$') AS FLOAT64) AS open,
      CAST(JSON_EXTRACT_SCALAR(r.high_array[OFFSET(i.pos)], '$') AS FLOAT64) AS high,
      CAST(JSON_EXTRACT_SCALAR(r.low_array[OFFSET(i.pos)], '$') AS FLOAT64) AS low,
      CAST(JSON_EXTRACT_SCALAR(r.close_array[OFFSET(i.pos)], '$') AS FLOAT64) AS close,
      CAST(JSON_EXTRACT_SCALAR(r.volume_array[OFFSET(i.pos)], '$') AS INT64) AS volume,
      
      -- ADX 관련 지표
      CASE 
        WHEN ARRAY_LENGTH(r.adx_14_values_array) > i.pos 
        THEN CAST(JSON_EXTRACT_SCALAR(r.adx_14_values_array[OFFSET(i.pos)], '$') AS FLOAT64)
        ELSE NULL 
      END AS adx_14,
      
      CASE 
        WHEN ARRAY_LENGTH(r.pdi_14_values_array) > i.pos 
        THEN CAST(JSON_EXTRACT_SCALAR(r.pdi_14_values_array[OFFSET(i.pos)], '$') AS FLOAT64)
        ELSE NULL 
      END AS pdi_14,
      
      CASE 
        WHEN ARRAY_LENGTH(r.mdi_14_values_array) > i.pos 
        THEN CAST(JSON_EXTRACT_SCALAR(r.mdi_14_values_array[OFFSET(i.pos)], '$') AS FLOAT64)
        ELSE NULL 
      END AS mdi_14,
      
      -- Chaikin Oscillator (수정된 필드명)
      CASE 
        WHEN ARRAY_LENGTH(r.chaikin_oscillator_array) > i.pos 
        THEN CAST(JSON_EXTRACT_SCALAR(r.chaikin_oscillator_array[OFFSET(i.pos)], '$') AS FLOAT64)
        ELSE NULL 
      END AS chaikin_oscillator,
      
      CASE 
        WHEN ARRAY_LENGTH(r.chaikin_signal_array) > i.pos 
        THEN CAST(JSON_EXTRACT_SCALAR(r.chaikin_signal_array[OFFSET(i.pos)], '$') AS FLOAT64)
        ELSE NULL 
      END AS chaikin_signal
      
    FROM raw_data r
    CROSS JOIN indices i
    WHERE i.ticker = r.ticker {date_filter}
    ORDER BY date DESC
    """
    
    try:
        df = client.query(query).to_dataframe()
        df['date'] = pd.to_datetime(df['date'])
        df.set_index('date', inplace=True)
        df.sort_index(inplace=True)
        
        # Chaikin의 전일 값 계산
        df['chaikin_yesterday'] = df['chaikin_oscillator'].shift(1)
        
        # ATR 계산 추가
        df['atr'] = calculate_atr(df)
        df['atr_ma'] = df['atr'].rolling(window=20).mean()
        
        # 절대 모멘텀 계산 (20일 수익률)
        df['momentum_20'] = df['close'].pct_change(periods=20)
        
        print(f"✅ {ticker} 데이터 로드 완료: {len(df)}개 레코드")
        return df
    except Exception as e:
        print(f"❌ 데이터 로드 실패: {e}")
        return None

## 2. 함수정의

### 함수1

In [19]:
import backtest_strategy as bts
help(bts.entry_strategies)


def test_entry_strategies(df, k=0.5):
    result = df.copy()
    result = bts.volatility_breakout_entry(result, k=k)
    return result


Help on module backtest_strategy.entry_strategies in backtest_strategy:

NAME
    backtest_strategy.entry_strategies

DESCRIPTION
    다양한 진입(Entry) 전략 구현
    변동성 돌파를 기반으로 한 여러 진입 조건들

FUNCTIONS
    adaptive_k_entry(df, lookback=20, k_min=0.3, k_max=0.7)
        적응형 K값 진입 전략
        시장 상황에 따라 K값을 동적으로 조정

        Parameters:
        - lookback: K값 최적화를 위한 과거 기간
        - k_min: 최소 K값
        - k_max: 최대 K값

    atr_filtered_entry(df, k=0.5, atr_period=14, min_atr_percentile=30)
        ATR 필터 진입 전략
        충분한 변동성이 있을 때만 진입

    composite_entry(df, k=0.5, min_score=3)
        복합 진입 전략
        여러 조건의 점수를 합산하여 진입

    double_breakout_entry(df, k1=0.5, k2=0.7, holding_period=5)
        이중 돌파 진입 전략
        1차 돌파 후 일정 기간 내 2차 돌파 시 진입

    gap_adjusted_entry(df, k=0.5, gap_threshold=0.02)
        갭 조정 진입 전략
        갭 상승/하락에 따라 진입가 조정

    momentum_filtered_entry(df, k=0.5, momentum_period=20, momentum_threshold=0.05)
        모멘텀 필터 진입 전략
        상승 모멘텀이 있을 때만 진입

    multi_timeframe_entry(df,

## 2. 데이터로드

In [6]:
# 개선된 전략 비교 함수 사용
from backtest_strategy import compare_all_entry_strategies, compare_all_exit_strategies
classic_etf_leverage = ['TQQQ','SQQQ','VDE','IWM','UBT','UGL','YCS','TBT']
start_date = '2020-05-29'

# 분석 대상 종목
tickers = classic_etf_leverage
start_date = '2015-07-01'

# ADX와 Chaikin 포함 데이터 로드
stock_data = {}
for ticker in tickers:
    df = get_stock_data_with_indicators(ticker, start_date)
    if df is not None and len(df) > 0:
        stock_data[ticker] = df
    else:
        print(f"⚠️ {ticker} 데이터 로드 실패 또는 비어있음")

print(f"\n✅ 총 {len(stock_data)}개 종목 데이터 로드 완료")

# 데이터 확인
if len(stock_data) == 0:
    print("❌ 로드된 데이터가 없습니다. BigQuery 연결을 확인해주세요.")
else:
    # 첫 번째 종목의 Chaikin 데이터 확인
    first_ticker = list(stock_data.keys())[0]
    test_df = stock_data[first_ticker]
    print(f"\n📊 {first_ticker}의 데이터 정보:")
    print(f"- 데이터 수: {len(test_df)}개")
    print(f"- 시작일: {test_df.index.min()}")
    print(f"- 종료일: {test_df.index.max()}")
    
    print(f"\n📊 {first_ticker}의 Chaikin 데이터 샘플:")
    print(test_df[['chaikin_oscillator', 'chaikin_signal']].head(10))
    
    # null이 아닌 Chaikin 데이터 수 확인
    chaikin_valid = test_df['chaikin_oscillator'].notna().sum()
    print(f"\n✅ {first_ticker}의 유효한 Chaikin 데이터: {chaikin_valid}개")
    
    # Chaikin 통계
    if chaikin_valid > 0:
        print(f"\n📊 {first_ticker}의 Chaikin 통계:")
        print(f"최소값: {test_df['chaikin_oscillator'].min():.2f}")
        print(f"최대값: {test_df['chaikin_oscillator'].max():.2f}")
        print(f"평균값: {test_df['chaikin_oscillator'].mean():.2f}")




✅ TQQQ 데이터 로드 완료: 2514개 레코드
✅ SQQQ 데이터 로드 완료: 2514개 레코드
✅ VDE 데이터 로드 완료: 2514개 레코드
✅ IWM 데이터 로드 완료: 2515개 레코드
✅ UBT 데이터 로드 완료: 2514개 레코드
✅ UGL 데이터 로드 완료: 2514개 레코드
✅ YCS 데이터 로드 완료: 2514개 레코드
✅ TBT 데이터 로드 완료: 2514개 레코드

✅ 총 8개 종목 데이터 로드 완료

📊 TQQQ의 데이터 정보:
- 데이터 수: 2514개
- 시작일: 2015-08-21 00:00:00
- 종료일: 2025-08-20 00:00:00

📊 TQQQ의 Chaikin 데이터 샘플:
            chaikin_oscillator  chaikin_signal
date                                          
2015-08-21       -1.000000e+00            -1.0
2015-08-24       -1.000000e+00            -1.0
2015-08-25       -1.000000e+00            -1.0
2015-08-26       -1.000000e+00            -1.0
2015-08-27       -1.000000e+00            -1.0
2015-08-28       -1.000000e+00            -1.0
2015-08-31       -1.000000e+00            -1.0
2015-09-01       -1.000000e+00            -1.0
2015-09-02       -1.000000e+00            -1.0
2015-09-03        4.387367e+07            -1.0

✅ TQQQ의 유효한 Chaikin 데이터: 2514개

📊 TQQQ의 Chaikin 통계:
최소값: -230598030.14
최대값: 289188892

### 3. 전략실행

In [20]:
# 전략 비교 실행
comparison_results = {}
all_results = {}  # 전체 결과 저장
all_tickers_data = stock_data  # 주가 데이터
selected_tickers = list(stock_data.keys())  # 모든 티커 사용

k_value = 0.3
adx_threshold = 35
best_k = 0.3  # 최적 K값 설정
momentum_thresholds = 0
momentum_period = 20
use_atr_filter=False
atr_period = 14
slippage=0
commission=0

# print(f"\n📊 ADX 필터 vs ADX/Chaikin 복합 필터 전략 비교 (K={k_value}, ADX={adx_threshold}):")
# print("=" * 120)
# print(f"{'티커':^10} | {'함수1':^35} | {'함수2':^35} | {'개선 효과':^20}")
# print(f"{'':^10} | {'수익률':^11}{'거래수':^10}{'승률':^10} | {'수익률':^11}{'거래수':^10}{'승률':^10} | {'수익률 차이':^10}{'거래수 차이':^10}")
# print("-" * 120)

for ticker, df in stock_data.items():
    # 모든 필터 조합에 대해 전략 테스트
    result = test_entry_strategies(df, k=k_value)
    print(result.keys())

Index(['open', 'high', 'low', 'close', 'volume', 'adx_14', 'pdi_14', 'mdi_14',
       'chaikin_oscillator', 'chaikin_signal', 'chaikin_yesterday', 'atr',
       'atr_ma', 'momentum_20', 'prev_range', 'target_price', 'entry_signal',
       'entry_price'],
      dtype='object')
Index(['open', 'high', 'low', 'close', 'volume', 'adx_14', 'pdi_14', 'mdi_14',
       'chaikin_oscillator', 'chaikin_signal', 'chaikin_yesterday', 'atr',
       'atr_ma', 'momentum_20', 'prev_range', 'target_price', 'entry_signal',
       'entry_price'],
      dtype='object')
Index(['open', 'high', 'low', 'close', 'volume', 'adx_14', 'pdi_14', 'mdi_14',
       'chaikin_oscillator', 'chaikin_signal', 'chaikin_yesterday', 'atr',
       'atr_ma', 'momentum_20', 'prev_range', 'target_price', 'entry_signal',
       'entry_price'],
      dtype='object')
Index(['open', 'high', 'low', 'close', 'volume', 'adx_14', 'pdi_14', 'mdi_14',
       'chaikin_oscillator', 'chaikin_signal', 'chaikin_yesterday', 'atr',
       'atr_ma'

In [21]:
# 진입 전략 비교
print("📊 진입 전략 비교 (익일 매도 기준)")
entry_df = compare_all_entry_strategies(df, k_values=[0.3, 0.5, 0.7])
print(entry_df)

# 청산 전략 비교
print("\n📊 청산 전략 비교 (변동성 돌파 K=0.5 기준)")
exit_df = compare_all_exit_strategies(df, entry_k=0.5)
print(exit_df)

📊 진입 전략 비교 (익일 매도 기준)
⚠️ 거래량 확인 실행 중 오류: cannot convert float NaN to bool
⚠️ 거래량 확인 실행 중 오류: cannot convert float NaN to bool
⚠️ 거래량 확인 실행 중 오류: cannot convert float NaN to bool
⚠️ 패턴 기반 실행 중 오류: cannot convert float NaN to bool
⚠️ 패턴 기반 실행 중 오류: cannot convert float NaN to bool
⚠️ 패턴 기반 실행 중 오류: cannot convert float NaN to bool
          전략  최적 K값  수익률(%)  거래횟수  승률(%)
2      이중 돌파    0.7  -72.98   204   42.6
4     모멘텀 필터    0.7  -91.83   111   33.3
8     ATR 필터    0.5  -93.43   174   30.5
7    멀티타임프레임    0.7  -94.43   135   32.6
0  기본 변동성 돌파    0.7  -99.92   367   27.0
5       갭 조정    0.7  -99.96   371   25.9
1     적응형 K값    0.7  -99.97   537   27.7
9      복합 점수    0.3  -99.99   809   31.9
3     거래량 확인    NaN    -inf     0    0.0
6      패턴 기반    NaN    -inf     0    0.0

📊 청산 전략 비교 (변동성 돌파 K=0.5 기준)
⚠️ 복합 점수 실행 중 오류: boolean value of NA is ambiguous
       전략  수익률(%)  거래횟수  평균보유일  승률(%)   샤프비율      최대낙폭
5   부분 익절  -72.66    33   36.0    0.0  -1.97   -72.66%
7  볼린저 밴드  -98.45   230    

In [22]:
# 개선된 백테스트 예시
# 간단한 백테스트 함수 사용
from backtest_strategy import simple_volatility_breakout_backtest, simple_backtest_entry_exit

# 1. 간단한 변동성 돌파 백테스트
result = simple_volatility_breakout_backtest(df, k=0.5, slippage=0.002, commission=0.001)

print(f"총 수익률: {(result['cumulative_returns'].iloc[-1] - 1) * 100:.2f}%")
print(f"Buy & Hold: {(result['buy_hold_returns'].iloc[-1] - 1) * 100:.2f}%")
print(f"거래 횟수: {result['num_trades'].iloc[0]}")
print(f"승률: {result['win_rate'].iloc[0] * 100:.1f}%")

# 2. 진입/청산 전략 조합
entry_result = volatility_breakout_entry(df, k=0.5)
entry_result.tail()
# exit_result = atr_based_exit(df, entry_result, take_profit_atr=2.0, stop_loss_atr=1.0)
# final_result = simple_backtest_entry_exit(df, entry_result, exit_result)

# print(f"\nATR 청산 전략:")
# print(f"총 수익률: {(final_result['cumulative_returns'].iloc[-1] - 1) * 100:.2f}%")
# print(f"거래 횟수: {final_result['total_trades'].iloc[0]}")

# # 수익률 그래프
# plt.figure(figsize=(12, 6))
# plt.plot(result.index, result['cumulative_returns'], label='기본 전략', linewidth=2)
# plt.plot(final_result.index, final_result['cumulative_returns'], label='ATR 청산', linewidth=2)
# plt.plot(result.index, result['buy_hold_returns'], label='Buy & Hold', linewidth=2, alpha=0.7)
# plt.legend()
# plt.title('전략 비교')
# plt.xlabel('날짜')
# plt.ylabel('누적 수익률')
# plt.grid(True, alpha=0.3)
# plt.show()

총 수익률: -99.68%
Buy & Hold: -0.15%
거래 횟수: 1075
승률: 35.5%


,open,high,low,close,volume,adx_14,pdi_14,mdi_14,chaikin_oscillator,chaikin_signal,chaikin_yesterday,atr,atr_ma,momentum_20,prev_range,target_price,entry_signal,entry_price
date,,,,,,,,,,,,,,,,,,
2025-08-14,35.470001,35.980000,35.470001,35.910000,670400,13.737307,28.151410,29.762419,-192778.946791,-269970.856098,-350981.719241,0.671428,0.708928,-0.044184,0.320000,35.630001,True,35.630001
2025-08-15,36.200001,36.540001,36.110001,36.400002,351300,13.280856,32.189789,27.783546,-71541.831133,-230285.051105,-192778.946791,0.677856,0.706964,-0.028297,0.509998,36.455000,True,36.455000
2025-08-18,36.320000,36.810001,36.320000,36.660000,334000,13.179160,33.398492,26.317858,25766.326768,-179074.775531,-71541.831133,0.623571,0.701500,-0.006773,0.430000,36.535000,True,36.535000
2025-08-19,36.419998,36.419998,36.150002,36.169998,212200,12.829444,31.533886,26.709512,5714.773414,-142116.865742,25766.326768,0.615714,0.694928,-0.007409,0.490002,36.664999,False,36.664999
2025-08-20,36.320000,36.320000,35.970001,36.099998,451500,12.232816,30.284318,27.689047,-39950.358171,-121683.564228,5714.773414,0.597857,0.688178,-0.019288,0.269997,36.454998,False,36.454998


In [23]:
# 청산 전략 비교 (변동성 돌파 K=0.5 기준)
exit_comparison = compare_exit_strategies(df, entry_k=0.5)

# 결과 요약 테이블
exit_data = []
for strategy_name, data in exit_comparison.items():
    exit_data.append({
        '전략': strategy_name,
        '총 수익률': f"{data['total_return'] * 100:.2f}%",
        '평균 보유일수': f"{data['avg_holding_days']:.1f}",
        '거래횟수': data['num_trades'],
        '샤프비율': data['metrics']['샤프비율'],
        '최대낙폭': data['metrics']['최대낙폭']
    })

exit_df = pd.DataFrame(exit_data)
print("\n📊 청산 전략 비교 (변동성 돌파 K=0.5 기준):")
print(exit_df.to_string(index=False))

TypeError: boolean value of NA is ambiguous

In [ ]:
# 상위 3개 청산 전략 선택
# 수익률 값을 float으로 변환하여 정렬
exit_df['수익률_수치'] = exit_df['총 수익률'].str.replace('%', '').astype(float)
top_strategies = exit_df.nlargest(3, '수익률_수치')['전략'].tolist()

# 선택된 전략들의 결과 딕셔너리 생성
strategies_to_plot = {}
for strategy_name in top_strategies:
    if strategy_name in exit_comparison:
        strategies_to_plot[strategy_name] = exit_comparison[strategy_name]['result']

# 시각화
if len(strategies_to_plot) > 0:
    fig = plot_strategy_comparison(
        strategies_to_plot,
        metrics_to_plot=['cumulative_returns', 'drawdown'],
        figsize=(15, 10)
    )
    plt.show()
else:
    print("시각화할 전략이 없습니다.")

## 4. 여러 진입 전략 비교

In [ ]:
# 진입 전략 비교 (익일 매도 기준)
entry_comparison = compare_entry_strategies(df, k_values=[0.3, 0.5, 0.7])

# 결과 요약 테이블
comparison_data = []
for strategy_name, data in entry_comparison.items():
    comparison_data.append({
        '전략': strategy_name,
        '최적 K값': data['best_k'],
        '총 수익률': f"{data['total_return'] * 100:.2f}%",
        '샤프비율': data['metrics']['샤프비율'],
        '최대낙폭': data['metrics']['최대낙폭'],
        '거래횟수': data['metrics']['거래횟수']
    })

comparison_df = pd.DataFrame(comparison_data)
print("\n📊 진입 전략 비교 (익일 매도 기준):")
print(comparison_df.to_string(index=False))

## 5. 여러 청산 전략 비교

In [ ]:
# 청산 전략 비교 (변동성 돌파 K=0.5 기준)
exit_comparison = compare_exit_strategies(df, entry_k=0.5)

# 결과 요약 테이블
exit_data = []
for strategy_name, data in exit_comparison.items():
    exit_data.append({
        '전략': strategy_name,
        '총 수익률': f"{data['total_return'] * 100:.2f}%",
        '평균 보유일수': f"{data['avg_holding_days']:.1f}",
        '거래횟수': data['num_trades'],
        '샤프비율': data['metrics']['샤프비율'],
        '최대낙폭': data['metrics']['최대낙폭']
    })

exit_df = pd.DataFrame(exit_data)
print("\n📊 청산 전략 비교 (변동성 돌파 K=0.5 기준):")
print(exit_df.to_string(index=False))

## 6. 전략 비교 시각화

In [ ]:
# 상위 3개 청산 전략 선택
top_strategies = exit_df.nlargest(3, '총 수익률')['전략'].tolist()

# 선택된 전략들의 결과 딕셔너리 생성
strategies_to_plot = {}
for strategy_name in top_strategies:
    strategies_to_plot[strategy_name] = exit_comparison[strategy_name]['result']

# 시각화
fig = plot_strategy_comparison(
    strategies_to_plot,
    metrics_to_plot=['cumulative_returns', 'drawdown'],
    figsize=(15, 10)
)
plt.show()

## 7. 맞춤형 전략 구성

In [ ]:
# 맞춤형 진입 조건: 거래량 확인 + 모멘텀 필터
# 1단계: 거래량 확인 진입
volume_entry = volume_confirmed_entry(df, k=0.5, volume_multiplier=1.5)

# 2단계: ATR 기반 청산
atr_exit = atr_based_exit(df, volume_entry, 
                         take_profit_atr=2.5,  # 더 넓은 익절선
                         stop_loss_atr=1.0,
                         max_holding_days=15)

# 백테스트 실행
custom_result = run_backtest(
    df,
    volume_confirmed_entry,
    atr_based_exit,
    entry_params={'k': 0.5, 'volume_multiplier': 1.5},
    exit_params={'take_profit_atr': 2.5, 'stop_loss_atr': 1.0, 'max_holding_days': 15},
    slippage=0.002,
    commission=0.001
)

# 성과 분석
custom_metrics = calculate_performance_metrics(custom_result['returns'])
print("\n📊 맞춤형 전략 성과:")
for key, value in custom_metrics.items():
    print(f"{key}: {value}")

# 청산 사유 분석
if 'exit_reason' in atr_exit.columns:
    exit_reasons = analyze_exit_reasons(atr_exit)
    print("\n📊 청산 사유 분석:")
    print(f"청산 사유별 횟수: {exit_reasons['exit_counts']}")
    print(f"청산 사유별 평균 수익률: {exit_reasons['exit_returns']}")

## 8. 실전 적용 예시

In [ ]:
# 실전에서 사용할 최종 전략
def my_trading_strategy(df):
    """
    나만의 트레이딩 전략
    - 진입: 적응형 K값 + 모멘텀 필터
    - 청산: ATR 기반 동적 손절/익절
    """
    # 진입 전략
    entry = momentum_filtered_entry(df, k=0.5, momentum_threshold=0.05)
    
    # 청산 전략  
    exit = atr_based_exit(df, entry,
                         take_profit_atr=2.0,
                         stop_loss_atr=1.0,
                         trailing_stop_atr=1.5,
                         max_holding_days=10)
    
    # 백테스트
    result = run_backtest(
        df,
        momentum_filtered_entry,
        atr_based_exit,
        entry_params={'k': 0.5, 'momentum_threshold': 0.05},
        exit_params={'take_profit_atr': 2.0, 'stop_loss_atr': 1.0, 
                    'trailing_stop_atr': 1.5, 'max_holding_days': 10},
        slippage=0.002,
        commission=0.001
    )
    
    return result

# 전략 실행
my_result = my_trading_strategy(df)

# 최종 성과
print(f"\n🎯 나만의 전략 최종 성과:")
print(f"총 수익률: {(my_result['cumulative_returns'].iloc[-1] - 1) * 100:.2f}%")
print(f"Buy & Hold 대비: {((my_result['cumulative_returns'].iloc[-1] / my_result['buy_hold_returns'].iloc[-1]) - 1) * 100:.2f}%")

# 월별 수익률
my_result['month'] = my_result.index.to_period('M')
monthly_returns = my_result.groupby('month')['returns'].apply(lambda x: (1 + x).prod() - 1)
print(f"\n월평균 수익률: {monthly_returns.mean() * 100:.2f}%")
print(f"수익 월 비율: {(monthly_returns > 0).mean() * 100:.1f}%")